In [ ]:
import csv
from random import randrange
from random import random
from random import seed
from math import sqrt
from math import exp
from math import tanh

In [ ]:
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = csv.reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset

In [ ]:
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())


In [ ]:
def str_column_to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    unique = set(class_values)
    lookup = dict()
    for i, value in enumerate(unique):
        lookup[value] = i
    for row in dataset:
        row[column] = lookup[row[column]]
    return lookup

In [ ]:
def dataset_minmax(dataset):
    minmax = list()
    for i in range(len(dataset[0])):
        colvalues = [row[i] for row in dataset]
        min_value = min(colvalues)
        max_value = max(colvalues)
        minmax.append([min_value, max_value])
    return minmax

In [ ]:
def Normalize_Dataset(dataset, minmax):
    for row in dataset:
        for i in range(len(row)-1):
            row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])

In [ ]:
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for _ in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

In [ ]:
def evaluate_algorithm_cv(dataset, algorithm, n_folds, performance_assessment, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        performance = performance_assessment(actual, predicted)
        scores.append(performance)
    return scores

In [ ]:
def getAccuracy(actual,predicted):
    correct = 0
    for x in range(len(actual)):
        if actual[x] == predicted[x]:
            correct += 1
    return correct / float(len(actual)) * 100.00

In [ ]:
def ini_network(n_inputs, n_hidden, n_outputs, added_sub_layers):
    network= list()
    hidden_layer_ini = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
    network.append(hidden_layer_ini)
    for _ in range(added_sub_layers):
        hidden_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_hidden)]
        network.append(hidden_layer)
    output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
    network.append(output_layer)
    return network

In [ ]:
def activate(weights, input):
    #Choosing final row of weights for the bias
    activation = weights[-1]
    for i in range(len(weights)-1):
        activation += weights[i]*input[i]
    return activation

In [ ]:
def transfer(activation, functiontype):
    if functiontype == 'Exponential':
        return 1.0 / (1.0 + exp(-activation))
    elif functiontype == 'Tanh':
        return tanh(activation)

In [ ]:
def forward_propagation(network, instance, activation_function):
    inputs = instance
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs)
            neuron['output'] = transfer(activation, activation_function)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    return inputs


In [ ]:
def transfer_derivative(output, activation_function):
    if activation_function == 'Exponential':
        return output * (1.0 - output)
    elif activation_function == 'Tanh':
        return 1 - (tanh(output))**2

In [ ]:
def backward_propagate_error(network, expected, activation_function):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        if i != len(network)-1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i+1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output'])ww
        for j in range(len(layer)):
                neuron = layer[j]
                neuron['delta'] = errors[j] * transfer_derivative(neuron['output'], activation_function)

In [ ]:
def update_weights(network, row, l_rate):
    for i in range(len(network)):
        #input all weights except last term which is the bias
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
            neuron['weights'][-1] += l_rate * neuron['delta']

In [ ]:
def train_network(network, train, l_rate, n_epoch, n_outputs, activation_function):
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            outputs = forward_propagation(network, row, activation_function)
            expected = [0 for i in range(n_outputs)]
            expected[row[-1]] = 1
            sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
            backward_propagate_error(network, expected, activation_function)
            update_weights(network, row, l_rate)
        print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))

In [ ]:
def predict(network, row, activation_function):
    outputs = forward_propagation(network, row, activation_function)
    return outputs.index(max(outputs))

In [ ]:
def back_propagation_sgs(train, test, l_rate, n_epoch, n_hidden, added_sub_layers):
    n_inputs = len(train[0]) - 1
    n_outputs = len(set([row[-1] for row in train]))
    network = ini_network(n_inputs, n_hidden, n_outputs, added_sub_layers)
    train_network(network, train, l_rate, n_epoch, n_outputs, 'Exponential')
    predictions = list()
    for row in test:
        prediction = predict(network, row, 'Exponential')
        predictions.append(prediction)
    return(predictions)

In [ ]:
from random import seed


In [ ]:
def main():
    # Test Backprop on Seeds dataset
    seed(1)
    # load and prepare data
    filename = 'wheat_seeds.csv'
    dataset = load_csv(filename)
    for i in range(len(dataset[0])-1):
        str_column_to_float(dataset, i)
    # convert class column to integers
    str_column_to_int(dataset, len(dataset[0])-1)
    # normalize input variables
    minmax = dataset_minmax(dataset)
    Normalize_Dataset(dataset, minmax)

    # evaluate algorithm
    n_folds = 10 # folds number for k fold cross-validation
    l_rate = 0.5 # learning rate
    n_epoch = 200 # number of epochs
    n_hidden = 10 # number of hidden nodes in hidden layers of ANN
    num_added_hidden_layers = 0 # num of hidden layers added (to the initial one)

    scores = evaluate_algorithm_cv(dataset, back_propagation_sgs, n_folds, getAccuracy,l_rate, n_epoch, n_hidden, num_added_hidden_layers)
    print('Scores: %s' % scores)
    print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

main()


>epoch=0, lrate=0.500, error=213.899
>epoch=1, lrate=0.500, error=127.672
>epoch=2, lrate=0.500, error=101.724
>epoch=3, lrate=0.500, error=78.205
>epoch=4, lrate=0.500, error=64.819
>epoch=5, lrate=0.500, error=54.804
>epoch=6, lrate=0.500, error=46.898
>epoch=7, lrate=0.500, error=41.335
>epoch=8, lrate=0.500, error=37.630
>epoch=9, lrate=0.500, error=35.087
>epoch=10, lrate=0.500, error=33.222
>epoch=11, lrate=0.500, error=31.760
>epoch=12, lrate=0.500, error=30.548
>epoch=13, lrate=0.500, error=29.498
>epoch=14, lrate=0.500, error=28.562
>epoch=15, lrate=0.500, error=27.711
>epoch=16, lrate=0.500, error=26.927
>epoch=17, lrate=0.500, error=26.201
>epoch=18, lrate=0.500, error=25.526
>epoch=19, lrate=0.500, error=24.896
>epoch=20, lrate=0.500, error=24.308
>epoch=21, lrate=0.500, error=23.759
>epoch=22, lrate=0.500, error=23.244
>epoch=23, lrate=0.500, error=22.760
>epoch=24, lrate=0.500, error=22.305
>epoch=25, lrate=0.500, error=21.874
>epoch=26, lrate=0.500, error=21.467
>epoch=2